In [ ]:
import pandas as pd
import numpy as np


df = pd.read_excel('cleaned_data.xlsx')


df['day_of_week'] = pd.to_datetime(df['BookingEndDateTime (Month / Day / Year)']).dt.day_name()


df['BookingStartTime'] = pd.to_datetime(df['BookingStartTime']).dt.time


def categorize_time(time):
    if 5 <= time.hour < 12:
        return 'Morning'
    elif 12 <= time.hour < 17:
        return 'Afternoon'
    else:
        return 'Evening'

df['time_of_day'] = df['BookingStartTime'].apply(categorize_time)

def get_dynamic_price(base_price, activity_description, time_of_day, current_bookings, max_bookees):
    """Calculates a dynamic price based on defined rules."""

    adjusted_price = base_price

    high_demand_classes = ['Body Conditioning 10-11am', 'Aquafit 7.00-8.00pm', 'Body Balance 11-12pm']
    if activity_description in high_demand_classes:
        adjusted_price *= 1.15
    else:
        adjusted_price *= 0.95

    if time_of_day == 'Morning' or time_of_day == 'Evening':
        adjusted_price *= 1.05
    elif time_of_day == 'Afternoon':
        adjusted_price *= 0.95

    demand_percentage = (current_bookings / max_bookees) if max_bookees > 0 else 0
    if demand_percentage > 0.80:
        adjusted_price *= 1.10
    elif demand_percentage < 0.30:
        adjusted_price *= 0.95

    return round(adjusted_price, 2)


df['Original Price'] = df['Price (INR)']
df['New Dynamic Price'] = 0.0

for index, row in df.iterrows():
    new_price = get_dynamic_price(
        base_price=row['Original Price'],
        activity_description=row['ActivityDescription'],
        time_of_day=row['time_of_day'],
        current_bookings=row['Number Booked'],
        max_bookees=row['MaxBookees']
    )
    df.loc[index, 'New Dynamic Price'] = new_price

print(df[['ActivityDescription', 'Original Price', 'New Dynamic Price', 'Number Booked', 'time_of_day']].head())

/tmp/ipython-input-3501919419.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['BookingStartTime'] = pd.to_datetime(df['BookingStartTime']).dt.time


       ActivityDescription  Original Price  New Dynamic Price  Number Booked  \
0  20-20-20  2.45pm-3.45pm             499             450.35             12   
1  20-20-20  2.45pm-3.45pm             499             450.35             15   
2  20-20-20  2.45pm-3.45pm             499             450.35             14   
3  20-20-20  2.45pm-3.45pm             499             450.35              9   
4  20-20-20  2.45pm-3.45pm             499             427.83              7   

  time_of_day  
0   Afternoon  
1   Afternoon  
2   Afternoon  
3   Afternoon  
4   Afternoon  
